### Importing the necessary Libraries

In [12]:
import dask
import json
from pandas.io.json import json_normalize
from tqdm import tqdm
import os
import dask.dataframe as dd
import pandas as pd

In [9]:
df = dd.read_csv('train.csv')

In [10]:
csv_path = "train.csv"

In [28]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 12 entries, channelGrouping to visitStartTime
dtypes: object(7), int64(4), uint64(1)

### Columns in Dataset

In [12]:
df.columns

Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')

### In the given dataset, there are 4 features which are in JSON format, they are - 'device', 'geoNetwork', 'totals' and 'trafficSource'
### In the below code, we have flatten the dataframe, in order to extract all the JSON Formatted features as separate columns.

In [10]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource'] # we are definig a list of json column names

    df = dd.read_csv(csv_path,
                     converters={
                         column: json.loads for column in JSON_COLUMNS}, # It will create JSON object for every json column
                     dtype={'fullVisitorId': 'str'})  # we are considering 'fullvisitor id as string'
                     #nrows=nrows)

    for column in tqdm(JSON_COLUMNS):
        column_as_df = json_normalize(df[column]) # json_normalize will return a flatten dataframe of json columns
        
        column_as_df.columns = ["{0}.{1}".format(column, subcolumn) for subcolumn in column_as_df.columns] # we are taking  
                                                                                                           # column names
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True) # we are dropping json column and
                                                                                            # and merging data frame with parsed columns
        
    print("Loaded {0}. Shape: {1}".format(os.path.basename(csv_path), df.shape))
    
    return df


In [14]:
ddf=load_df(csv_path)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]C:\Users\dhira\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()
 25%|█████████████████████                                                               | 1/4 [01:38<04:55, 98.63s/it]C:\Users\dhira\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()
 50%|█████████████████████████████████████████▌                                         | 2/4 [03:33<03:36, 108.23s/it]C:\Users\dhira\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveS

Loaded train.csv. Shape: (Delayed('int-9972cece-a831-47ef-8e02-d90450d8d621'), 55)


In [15]:
ddf.head(2)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,...,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent,trafficSource.campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Identifying Columns with unique Value

In [16]:
l=[]
for i in list(ddf.columns):
    if len(list(ddf[i].unique()))==1:
        print(i)
        l.append(i)

socialEngagementType
device.browserVersion
device.browserSize
device.operatingSystemVersion
device.mobileDeviceBranding
device.mobileDeviceModel
device.mobileInputSelector
device.mobileDeviceInfo
device.mobileDeviceMarketingName
device.flashVersion
device.language
device.screenColors
device.screenResolution
geoNetwork.cityId
geoNetwork.latitude
geoNetwork.longitude
geoNetwork.networkLocation
totals.visits
trafficSource.adwordsClickInfo.criteriaParameters
trafficSource.campaignCode


In [41]:
len(l)

20

### Dropping all the columns with unique values

In [37]:
for i in l:
    ddf=ddf.drop(i,axis=1)

In [18]:
ddf.columns # Sanity Check if the columns dropped

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.operatingSystem', 'device.isMobile', 'device.deviceCategory',
       'geoNetwork.continent', 'geoNetwork.subContinent', 'geoNetwork.country',
       'geoNetwork.region', 'geoNetwork.metro', 'geoNetwork.city',
       'geoNetwork.networkDomain', 'totals.hits', 'totals.pageviews',
       'totals.bounces', 'totals.newVisits', 'totals.transactionRevenue',
       'trafficSource.campaign', 'trafficSource.source',
       'trafficSource.medium', 'trafficSource.keyword',
       'trafficSource.isTrueDirect', 'trafficSource.referralPath',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.adContent'],
      dtype='object')

### Calculating the Null values

#### Reference -  https://stackoverflow.com/questions/51070985/find-out-the-percentage-of-missing-values-in-each-column-in-the-given-dataset - Used a slight modified version of it

In [30]:
dt=(ddf.isnull().sum()/len(ddf.index))*100
missing_values=pd.DataFrame({'Columns':dt.index,'Null Values Count':dt.values})

In [31]:
missing_values

,Columns,Null Values Count
0,channelGrouping,0.0
1,date,0.0
2,fullVisitorId,0.0
3,sessionId,0.0
4,visitId,0.0
5,visitNumber,0.0
6,visitStartTime,0.0
7,device.browser,0.0
8,device.operatingSystem,0.0
9,device.isMobile,0.0


### Removing all the columns which have missing values greater than 90%

In [33]:
for i in missing_values.index:
    if missing_values.iloc[i]['Null Values Count']>90 and  missing_values.iloc[i]['Columns']!='totals.transactionRevenue':
        ddf=ddf.drop( missing_values.iloc[i]['Columns'] ,axis=1)

### The columns left after removing all columns with missing values > 90%

In [34]:
ddf.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.operatingSystem', 'device.isMobile', 'device.deviceCategory',
       'geoNetwork.continent', 'geoNetwork.subContinent', 'geoNetwork.country',
       'geoNetwork.region', 'geoNetwork.metro', 'geoNetwork.city',
       'geoNetwork.networkDomain', 'totals.hits', 'totals.pageviews',
       'totals.bounces', 'totals.newVisits', 'totals.transactionRevenue',
       'trafficSource.campaign', 'trafficSource.source',
       'trafficSource.medium', 'trafficSource.keyword',
       'trafficSource.isTrueDirect', 'trafficSource.referralPath'],
      dtype='object')

In [1]:
len(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.operatingSystem', 'device.isMobile', 'device.deviceCategory',
       'geoNetwork.continent', 'geoNetwork.subContinent', 'geoNetwork.country',
       'geoNetwork.region', 'geoNetwork.metro', 'geoNetwork.city',
       'geoNetwork.networkDomain', 'totals.hits', 'totals.pageviews',
       'totals.bounces', 'totals.newVisits', 'totals.transactionRevenue',
       'trafficSource.campaign', 'trafficSource.source',
       'trafficSource.medium', 'trafficSource.keyword',
       'trafficSource.isTrueDirect', 'trafficSource.referralPath'])

29

In [2]:
55-20-6

29

### Saving the CSV file

In [36]:
ddf.to_csv('cleaned_data.csv',index=False,single_file=True)

['C:/Users/dhira/Desktop/Coursera/Applied/Case Study - Google Anayltics/cleaned_data.csv']

In [39]:
len(ddf.columns)

29